In [1]:
# query = [
#   ["PROJECTION", ["name"]],
#   ["SELECTION", ["id", "EQUALS", "5000"]],
#   ["FILESCAN", ["movies"]]
# ]

In [2]:
class Node:
    def __init__(self, *args, child=None):
        self.args = args
        self.child = child
        self.schema = None

In [3]:
class Filescan(Node):
    def __iter__(self):
        table, = self.args
        path = f'ml-20m/{table}.csv'
        
        with open(path) as f:
            columns = next(f).strip().split(',')
            self.schema = dict(zip(columns, range(len(columns))))
            for line in f:
                yield line.strip().split(',')

In [4]:
class Selection(Node):
    def __iter__(self):
        column, op, value = self.args
        
        for row in self.child:
            if not self.schema:
                self.schema = self.child.schema
            if op(row[self.child.schema[column]], value):
                yield row

In [5]:
class Projection(Node):
    def __iter__(self):
        self.schema = dict(zip(self.args, range(len(self.args))))
        
        for row in self.child:
            yield tuple([row[self.child.schema[column]] for column in self.schema])

In [6]:
EQUALS = lambda x, y: x == y

In [7]:
CONTAINS = lambda string, substring: substring in string

In [24]:
q = Filescan('movies')
q = Selection('genres', CONTAINS, 'War', child=q)
q = Projection('genres', child=q)
q = Selection('genres', CONTAINS, 'Sci-Fi', child=q)

In [25]:
for result in q:
    print(result)

('Action|Sci-Fi|War',)
('Animation|Drama|Sci-Fi|War',)
('Comedy|Drama|Sci-Fi|War',)
('Action|Fantasy|Sci-Fi|Thriller|War',)
('Action|Drama|Sci-Fi|War',)
('Drama|Sci-Fi|War',)
('Action|Horror|Sci-Fi|War',)
('Animation|Fantasy|Sci-Fi|War',)
('Drama|Sci-Fi|War',)
('Fantasy|Mystery|Sci-Fi|War',)
('Action|Sci-Fi|War',)
('Action|Adventure|Sci-Fi|War',)
('Adventure|Comedy|Sci-Fi|Thriller|War',)
('Action|Sci-Fi|War',)
('Action|Adventure|Sci-Fi|Thriller|War',)
('Action|Adventure|Sci-Fi|War|IMAX',)
('Action|Adventure|Sci-Fi|Thriller|War',)
('Action|Sci-Fi|War',)
('Sci-Fi|Thriller|War',)
('Action|Adventure|Horror|Sci-Fi|Thriller|War',)
('Action|Animation|Sci-Fi|War',)
('Adventure|Animation|Drama|Sci-Fi|War',)
('Adventure|Animation|Drama|Sci-Fi|War',)
('Drama|Sci-Fi|War',)
('Action|Animation|Sci-Fi|War',)
('Animation|Comedy|Sci-Fi|War',)
('Drama|Horror|Sci-Fi|War',)
